<a href="https://colab.research.google.com/github/mike3071/Weather_Big_Data_Contest/blob/main/clf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

머신러닝 - 연령별/성별 분류  
정형데이터이고 분류이기 때문에 LGBM 사용  

In [1]:
import pandas as pd
import numpy as np

In [3]:
X_Rnn = pd.read_csv('./data/X_Rnn.csv',encoding='cp949')
y_Rnn = pd.read_csv('./data/y_Rnn.csv',encoding='cp949')

In [10]:
X_Rnn.head()

,날짜,소분류_가스온수기,소분류_가열식 가습기,소분류_감/홍시,소분류_감귤/한라봉/오렌지,소분류_감말랭이,소분류_감자,소분류_건대추,소분류_건포도,소분류_견과류 밤,소분류_계란,소분류_고추/피망/파프리카,소분류_곡물차,소분류_공기정화 용품,소분류_공기청정기,소분류_과일류,소분류_과일채소 분말/분태,소분류_굴 생물,소분류_기능성 아이케어 화장품,소분류_기능성 화이트닝 화장품,소분류_기초 화장용 로션,소분류_기초 화장용 크림,소분류_기타 주스류,소분류_김치류,소분류_난방용 열풍기,소분류_남성 로션,소분류_남성 선케어,소분류_남성 세트,소분류_남성향수,소분류_냉풍기,소분류_네일 메이크업 용품,소분류_다이어트용 헬스보충식품,소분류_대게/킹크랩,소분류_더치커피,소분류_데오드란트,소분류_돈풍기,소분류_둥굴레차,소분류_딸기/복분자/블루베리,소분류_라디에이터,소분류_레몬/자몽,...,소분류_입욕제,소분류_자연식 가습기,소분류_장어,소분류_전기온수기,소분류_전기장판,소분류_전통차,소분류_절임배추/김치속,소분류_제습기,소분류_조개,소분류_중대형 에어컨,소분류_참외/메론/수박,소분류_천장형 에어컨,소분류_초음파식 가습기,소분류_카페 푸드,소분류_카페트매트,소분류_컨벡터,소분류_코코아/핫초코,소분류_키위/참다래,소분류_탁상/USB 선풍기,소분류_탄산수,소분류_탄산음료,소분류_토마토,소분류_풋크림,소분류_해초류,소분류_핸드크림,소분류_향수세트,소분류_허브차,소분류_헤어스타일링용 흑채,소분류_홍차,소분류_황토매트,소분류_회,소분류_휴대용 선풍기,소분류_히터,평균기온(°C),최저기온(°C),최고기온(°C),최소 상대습도(%),평균 상대습도(%),O3,PM10
0,2018-01-01,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.066157,-3.463993,4.734708,20.763179,41.267581,0.015544,44.032069
1,2018-01-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.066157,-3.463993,4.734708,20.763179,41.267581,0.015544,44.032069
2,2018-01-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.066157,-3.463993,4.734708,20.763179,41.267581,0.015544,44.032069
3,2018-01-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.066157,-3.463993,4.734708,20.763179,41.267581,0.015544,44.032069
4,2018-01-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.066157,-3.463993,4.734708,20.763179,41.267581,0.015544,44.032069


In [4]:
y_Rnn.drop(['일별 물품 선호도 점수'],axis=1,inplace=True)
X_Rnn.drop(['소분류_여성향수'],axis=1,inplace=True)
df = pd.concat([X_Rnn,y_Rnn],axis=1)

In [5]:
df.rename(columns = {'20대 남성 선호도 점수':0,
                    '20대 여성 선호도 점수':1,
                    '30대 남성 선호도 점수':2,
                    '30대 여성 선호도 점수':3,
                    '40대 남성 선호도 점수':4,
                    '40대 여성 선호도 점수':5,
                    '50대 남성 선호도 점수':6,
                    '50대 여성 선호도 점수':7,
                    '60대 남성 선호도 점수':8,
                    '60대 여성 선호도 점수':9},inplace=True)

In [6]:
new_df = df[[0,1,2,3,4,5,6,7,8,9]]
new_score = new_df.idxmax(axis=1)
new_score1 = pd.DataFrame(new_score,columns=['score'])
fin_df = pd.concat([df,new_score1],axis=1)
fin_df.drop([0,1,2,3,4,5,6,7,8,9],axis=1,inplace=True)
fin1 = fin_df.copy()
fin2 = fin1.drop('날짜',axis=1)

In [8]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split

X = fin2.drop('score',axis=1).reset_index(drop=True)
y = fin2['score'].reset_index(drop=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0)
lgb_clf = LGBMClassifier(random_state=0)
lgb_clf.fit(X_train,y_train)
lgb_pred = lgb_clf.predict(X_test)

In [9]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, lgb_pred))

0.966411643963426


일별 물품 선호도 점수와 여성향수만 없앴을 때 accuracy : 0.966411643963426  
날짜를 월,일,요일로 나눈 후 rf acc = 0.8856751881569944  
안나누고 구한 rf acc = 0.8902780369471917 이므로 날짜는 사용하지 않음

In [11]:
X_Rnn.columns

Index(['날짜', '소분류_가스온수기', '소분류_가열식 가습기', '소분류_감/홍시', '소분류_감귤/한라봉/오렌지',
       '소분류_감말랭이', '소분류_감자', '소분류_건대추', '소분류_건포도', '소분류_견과류 밤',
       ...
       '소분류_회', '소분류_휴대용 선풍기', '소분류_히터', '평균기온(°C)', '최저기온(°C)', '최고기온(°C)',
       '최소 상대습도(%)', '평균 상대습도(%)', 'O3', 'PM10'],
      dtype='object', length=135)

In [17]:
y.value_counts()

8    13740
9    10087
0     9751
6     9627
1     8240
2     7512
4     7041
7     6097
3     4459
5     3830
Name: score, dtype: int64

In [25]:
# 정규화(날씨 변수들)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
new_x = X[[ '평균기온(°C)', '최저기온(°C)', '최고기온(°C)', '최소 상대습도(%)', '평균 상대습도(%)', 'O3', 'PM10']]
x_scaled = scaler.fit_transform(new_x)

In [26]:
x_scaled_df = pd.DataFrame(x_scaled,columns=['평균기온(°C)', '최저기온(°C)', '최고기온(°C)', '최소 상대습도(%)', '평균 상대습도(%)', 'O3', 'PM10'])

In [27]:
x_scaled_df.head()

,평균기온(°C),최저기온(°C),최고기온(°C),최소 상대습도(%),평균 상대습도(%),O3,PM10
0,0.270008,0.270049,0.278985,0.126538,0.210722,0.169762,0.250376
1,0.270008,0.270049,0.278985,0.126538,0.210722,0.169762,0.250376
2,0.270008,0.270049,0.278985,0.126538,0.210722,0.169762,0.250376
3,0.270008,0.270049,0.278985,0.126538,0.210722,0.169762,0.250376
4,0.270008,0.270049,0.278985,0.126538,0.210722,0.169762,0.250376


In [28]:
X1 = X.drop(['평균기온(°C)', '최저기온(°C)', '최고기온(°C)', '최소 상대습도(%)', '평균 상대습도(%)', 'O3', 'PM10'],axis=1)

In [29]:
new_df = pd.concat([X1,x_scaled_df],axis=1)

In [30]:
new_df.head()

,소분류_가스온수기,소분류_가열식 가습기,소분류_감/홍시,소분류_감귤/한라봉/오렌지,소분류_감말랭이,소분류_감자,소분류_건대추,소분류_건포도,소분류_견과류 밤,소분류_계란,소분류_고추/피망/파프리카,소분류_곡물차,소분류_공기정화 용품,소분류_공기청정기,소분류_과일류,소분류_과일채소 분말/분태,소분류_굴 생물,소분류_기능성 아이케어 화장품,소분류_기능성 화이트닝 화장품,소분류_기초 화장용 로션,소분류_기초 화장용 크림,소분류_기타 주스류,소분류_김치류,소분류_난방용 열풍기,소분류_남성 로션,소분류_남성 선케어,소분류_남성 세트,소분류_남성향수,소분류_냉풍기,소분류_네일 메이크업 용품,소분류_다이어트용 헬스보충식품,소분류_대게/킹크랩,소분류_더치커피,소분류_데오드란트,소분류_돈풍기,소분류_둥굴레차,소분류_딸기/복분자/블루베리,소분류_라디에이터,소분류_레몬/자몽,소분류_마/야콘,...,소분류_천장형 에어컨,소분류_초음파식 가습기,소분류_카페 푸드,소분류_카페트매트,소분류_컨벡터,소분류_코코아/핫초코,소분류_키위/참다래,소분류_탁상/USB 선풍기,소분류_탄산수,소분류_탄산음료,소분류_토마토,소분류_풋크림,소분류_해초류,소분류_핸드크림,소분류_향수세트,소분류_허브차,소분류_헤어스타일링용 흑채,소분류_홍차,소분류_황토매트,소분류_회,소분류_휴대용 선풍기,소분류_히터,20대 남성 판매량(개),20대 여성 판매량(개),30대 남성 판매량(개),30대 여성 판매량(개),40대 남성 판매량(개),40대 여성 판매량(개),50대 남성 판매량(개),50대 여성 판매량(개),60대 남성 판매량(개),60대 여성 판매량(개),합계,평균기온(°C),최저기온(°C),최고기온(°C),최소 상대습도(%),평균 상대습도(%),O3,PM10
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,0.0,0.0,2.0,3.0,0.0,0.0,0.0,0.0,7.0,0.270008,0.270049,0.278985,0.126538,0.210722,0.169762,0.250376
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7.0,12.0,6.0,18.0,6.0,8.0,3.0,2.0,0.0,1.0,63.0,0.270008,0.270049,0.278985,0.126538,0.210722,0.169762,0.250376
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,9.0,3.0,7.0,3.0,10.0,2.0,4.0,0.0,0.0,39.0,0.270008,0.270049,0.278985,0.126538,0.210722,0.169762,0.250376
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,4.0,5.0,1.0,3.0,6.0,3.0,5.0,0.0,1.0,30.0,0.270008,0.270049,0.278985,0.126538,0.210722,0.169762,0.250376
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,52.0,225.0,200.0,459.0,107.0,219.0,34.0,52.0,5.0,9.0,1362.0,0.270008,0.270049,0.278985,0.126538,0.210722,0.169762,0.250376


In [31]:
# 다중공선성 우려 합계 삭제
new_x_df = new_df.drop(['합계'],axis=1)

In [37]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(new_x_df, y, test_size=0.2,random_state=0)
lgb_clf1 = LGBMClassifier(random_state=0)
lgb_clf1.fit(X_train1,y_train1)
lgb_pred1 = lgb_clf1.predict(X_test1)

In [38]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test1, lgb_pred1))

0.9646700255022703


'합계' 삭제, 날씨 변수 정규화(MinMax) 후 train-test set 분할, LGBM 사용 accuracy는 0.9646700255022703

In [ ]:
new = pd.concat([new_x_df,y],axis=1)
new.head()

In [43]:
import random

In [44]:
# 교차검증
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
folds=[]
for train_idx, valid_idx in skf.split(new, new['score']):
    folds.append((train_idx, valid_idx))

random.seed(0)
lgb_models={}
for fold in range(5):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = new.drop(['score'],axis=1).iloc[train_idx].values, new.drop(['score'],axis=1).iloc[valid_idx].values,\
                                         new['score'][train_idx].values, new['score'][valid_idx].values 
    lgb = LGBMClassifier(n_estimators=1000)
    lgb.fit(X_train, y_train, 
            eval_set=[(X_train, y_train), (X_valid, y_valid)], 
            early_stopping_rounds=30,
           verbose=100)
    lgb_models[fold]=lgb
    print(f'================================================================================\n\n')

====================================1============================================
Training until validation scores don't improve for 30 rounds.
[100]	training's multi_logloss: 0.121536	valid_1's multi_logloss: 0.161101
[200]	training's multi_logloss: 0.0402715	valid_1's multi_logloss: 0.0804012
[300]	training's multi_logloss: 0.0166236	valid_1's multi_logloss: 0.0586546
[400]	training's multi_logloss: 0.00737058	valid_1's multi_logloss: 0.0506017
[500]	training's multi_logloss: 0.00337706	valid_1's multi_logloss: 0.0473739
Early stopping, best iteration is:
[546]	training's multi_logloss: 0.00237979	valid_1's multi_logloss: 0.0469504


====================================2============================================
Training until validation scores don't improve for 30 rounds.
[100]	training's multi_logloss: 0.122768	valid_1's multi_logloss: 0.161124
[200]	training's multi_logloss: 0.0404909	valid_1's multi_logloss: 0.0803411
[300]	training's multi_logloss: 0.0167492	valid_1's multi_lo

valid_multi_logloss 5-fold mean: 0.047647459999999996